<a href="https://colab.research.google.com/github/anirudh-hegde/myrmexai/blob/master/Ant_species_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow -y
!pip uninstall keras -y
!pip uninstall tf-keras -y


Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Successfully uninstalled keras-3.5.0
Found existing installation: tf_keras 2.17.0
Uninstalling tf_keras-2.17.0:
  Successfully uninstalled tf_keras-2.17.0


In [ ]:
!pip install tensorflow==2.17.0
# !pip install keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
# import keras
import random
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
from tensorflow.keras.models import load_model

In [ ]:

random.seed(111)
np.random.seed(111)
tf.random.set_seed(111)

In [ ]:
import os
dirs = os.listdir('/content/drive/MyDrive/Ant-Species/')
for dir in dirs:
  print(dir, len(os.listdir('/content/drive/MyDrive/Ant-Species/'+dir)))

fireant 813
ghostant 910
weaverant 947


In [ ]:
base_dir = "/content/drive/MyDrive/Ant-Species"
img_size = 180
batch_size = 28

In [ ]:

train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=111,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

Found 2670 files belonging to 3 classes.
Using 2136 files for training.


In [ ]:
valid_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=111,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

Found 2670 files belonging to 3 classes.
Using 534 files for validation.


In [ ]:
ant_names = train_ds.class_names
print(ant_names)

['fireant', 'ghostant', 'weaverant']


In [ ]:
train_ds = train_ds.cache().shuffle(1000, seed=111).prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
data_augmentation = Sequential([
  tf.keras.layers.RandomFlip('horizontal', input_shape=(img_size, img_size, 3)),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model = Sequential([
  data_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  Dropout(0.2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(len(ant_names))
])


In [ ]:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=20,
    batch_size=batch_size,
    # callbacks=[save_ck]
)


test_loss, valid_accuracy = model.evaluate(valid_ds)
print(f"Validation Accuracy: {valid_accuracy * 100:.4f}%")

Epoch 1/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.5299 - loss: 1.0332 - val_accuracy: 0.7041 - val_loss: 0.7337
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 60s 26ms/step - accuracy: 0.7403 - loss: 0.6488 - val_accuracy: 0.7472 - val_loss: 0.6811
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7579 - loss: 0.5805 - val_accuracy: 0.7772 - val_loss: 0.5770
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7923 - loss: 0.5160 - val_accuracy: 0.7453 - val_loss: 0.7318
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.8037 - loss: 0.5092 - val_accuracy: 0.8109 - val_loss: 0.5134
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8181 - loss: 0.4663 - val_accuracy: 0.8221 - val_loss: 0.4625
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8281 - loss: 0.4275 - val_accuracy: 0.8258 - val_loss: 0.4878
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8369 - loss: 0.4010 - val_accuracy: 0.8258 - 

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 180, 180, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 90, 90, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 90, 90, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 45, 45, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 45, 45, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 30976)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,965,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,967,083 (45.65 MB)

 Trainable params: 3,989,027 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,978,056 (30.43 MB)

In [ ]:
model.save("myrmex.h5")

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 180, 180, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 90, 90, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 90, 90, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 45, 45, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 45, 45, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 22, 22, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 30976)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,965,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,967,083 (45.65 MB)

 Trainable params: 3,989,027 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,978,056 (30.43 MB)

In [ ]:
input_image = tf.keras.utils.load_img('/content/drive/MyDrive/Ant-Species/ghostant/ghost-ant-10-_jpg.rf.7e281fa895587e4618da9405a3c69f1b.jpg', target_size=(img_size, img_size))
input_image_array = tf.keras.utils.img_to_array(input_image)
input_image_exp = tf.expand_dims(input_image_array, 0)
predictions = model.predict(input_image_exp)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step


array([[ 1.6203148,  5.6226006, -2.4625764]], dtype=float32)

In [ ]:
probabilities = tf.nn.softmax(predictions).numpy()
probabilities

array([[1.7940450e-02, 9.8175710e-01, 3.0245204e-04]], dtype=float32)

In [ ]:
res = np.argmax(probabilities)
res

1

In [ ]:
# predict_class = np.max(probabilities)
confidence = probabilities[0][res]
print(f"The uploaded image belongs to  {ant_names[np.argmax(probabilities)]} species with acuuracy of {confidence}")

The uploaded image belongs to  ghostant species with acuuracy of 0.9817571043968201


In [ ]:
model.save("myrmex.h5")

In [ ]:
ant_model = load_model("myrmex.h5")

In [ ]:
input_image = tf.keras.utils.load_img('/content/drive/MyDrive/Ant-Species/ghostant/ghost-ant-10-_jpg.rf.7e281fa895587e4618da9405a3c69f1b.jpg', target_size=(img_size, img_size))
input_image_array = tf.keras.utils.img_to_array(input_image)
input_image_exp = tf.expand_dims(input_image_array, 0)
predictions = ant_model.predict(input_image_exp)
res = np.argmax(predictions)
print(f"The uploaded image belongs to {ant_names[res]} species with acuuracy of {confidence}" )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
The uploaded image belongs to ghostant species with acuuracy of 0.9817571043968201
